Objective: Perform a second round of preprocessing with a focus towards

1. removing duplicate rows
2. removing irrelevant features
3. removing highly correlated features
4. remove outliers

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn.feature_selection import VarianceThreshold

In [2]:
pd.options.display.max_columns = None

In [3]:
df = pd.read_csv('/Users/jordonez/Desktop/Brainstation/CapstoneRepo/data/checkpoint4.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(columns=['date'])

In [4]:
# Drop duplicate rows
df_clean = df.drop_duplicates()

In [5]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5420909 entries, 0 to 5420908
Data columns (total 57 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   median_sale_price                        float64
 1   median_list_price                        float64
 2   median_ppsf                              float64
 3   median_list_ppsf                         float64
 4   homes_sold                               float64
 5   pending_sales                            float64
 6   new_listings                             float64
 7   inventory                                float64
 8   months_of_supply                         float64
 9   median_dom                               float64
 10  avg_sale_to_list                         float64
 11  sold_above_list                          float64
 12  price_drops                              float64
 13  off_market_in_two_weeks                  float64
 14  year              

In [13]:
# Remove features with low variance (close to zero)
selector = VarianceThreshold(threshold=0.1)
df_reduced = pd.DataFrame(selector.fit_transform(df_clean), columns=df_clean.columns[selector.get_support()])

In [14]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5420909 entries, 0 to 5420908
Data columns (total 42 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   median_sale_price                        float64
 1   median_list_price                        float64
 2   median_ppsf                              float64
 3   median_list_ppsf                         float64
 4   homes_sold                               float64
 5   pending_sales                            float64
 6   new_listings                             float64
 7   inventory                                float64
 8   months_of_supply                         float64
 9   median_dom                               float64
 10  year                                     float64
 11  month                                    float64
 12  quarter                                  float64
 13  season                                   float64
 14  price_diff        

In [15]:
# remove highly correlated features
# Calculate correlation matrix
corr_matrix = df_reduced.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of features with correlation greater than 0.9
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

# Drop those features
df_reduced = df_reduced.drop(to_drop, axis=1)

In [16]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5420909 entries, 0 to 5420908
Data columns (total 35 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   median_sale_price                        float64
 1   median_list_price                        float64
 2   median_ppsf                              float64
 3   median_list_ppsf                         float64
 4   homes_sold                               float64
 5   inventory                                float64
 6   months_of_supply                         float64
 7   median_dom                               float64
 8   year                                     float64
 9   month                                    float64
 10  season                                   float64
 11  sale_to_list_ratio                       float64
 12  price_growth                             float64
 13  buyer_utility                            float64
 14  pending_sales_rati

In [17]:
df_reduced.sample()

,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,inventory,months_of_supply,median_dom,year,month,season,sale_to_list_ratio,price_growth,buyer_utility,pending_sales_ratio,sales_success_rate,inventory_turnover,adjusted_months_supply,political_stance_encoded,supply_to_list_ratio,property_type_All Residential,property_type_Single Family Residential,state_avg_sale_price,metro_region_inventory_change,inventory_to_pending_ratio,rolling_median_sale_price,rolling_median_list_price,price_momentum,supply_pressure,demand_pressure,price_elasticity,us_region_East North Central,us_region_Mid-Atlantic,us_region_Pacific,us_region_South Atlantic
2050445,147381.0,123825.0,119.63,75.98,2.0,4.0,2.0,3.0,2017.0,4.0,1.0,1.190236,0.079714,0.159831,6.282934,2.0,0.5,1.0,1.0,4.0,0.0,0.0,145370.919227,-0.862069,0.636645,146497.0,146549.0,0.024119,0.5,1.570734,-32.478099,1.0,0.0,0.0,0.0


In [18]:
# remove outliers from data
z_scores = np.abs(stats.zscore(df_reduced))
threshold = 3  # z-scores greater than 3 are considered outliers

# Keep only the rows that don't have outliers
df_no_outliers = df_reduced[(z_scores < threshold).all(axis=1)]

In [19]:
df_no_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4529592 entries, 0 to 5420908
Data columns (total 35 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   median_sale_price                        float64
 1   median_list_price                        float64
 2   median_ppsf                              float64
 3   median_list_ppsf                         float64
 4   homes_sold                               float64
 5   inventory                                float64
 6   months_of_supply                         float64
 7   median_dom                               float64
 8   year                                     float64
 9   month                                    float64
 10  season                                   float64
 11  sale_to_list_ratio                       float64
 12  price_growth                             float64
 13  buyer_utility                            float64
 14  pending_sales_ratio    

In [20]:
len(df_no_outliers)

4529592

In [21]:
df_no_outliers.describe()

,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,inventory,months_of_supply,median_dom,year,month,season,sale_to_list_ratio,price_growth,buyer_utility,pending_sales_ratio,sales_success_rate,inventory_turnover,adjusted_months_supply,political_stance_encoded,supply_to_list_ratio,property_type_All Residential,property_type_Single Family Residential,state_avg_sale_price,metro_region_inventory_change,inventory_to_pending_ratio,rolling_median_sale_price,rolling_median_list_price,price_momentum,supply_pressure,demand_pressure,price_elasticity,us_region_East North Central,us_region_Mid-Atlantic,us_region_Pacific,us_region_South Atlantic
count,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06,4.529592e+06
mean,2.387273e+05,2.664266e+05,1.365971e+02,1.499583e+02,1.178471e+01,3.687788e+01,3.776697e+00,8.009395e+01,2.018128e+03,6.490458e+00,1.529686e+00,9.577192e-01,3.598121e-01,-4.195256e-01,1.080247e+00,8.794243e-01,4.490001e-01,1.624988e+00,1.106123e+00,2.900347e+00,3.746715e-01,3.779956e-01,2.591567e+05,3.590568e+00,4.983802e+00,2.445320e+05,2.727548e+05,8.962581e-01,2.545360e+00,6.599691e-01,-2.485473e-01,1.718122e-01,1.906772e-01,1.317324e-01,1.860552e-01
std,1.979347e+05,2.187787e+05,1.162793e+02,2.238964e+02,2.026396e+01,6.186655e+01,2.905825e+00,8.494433e+01,3.463318e+00,3.357114e+00,1.093562e+00,4.954399e-01,4.431294e+00,4.240309e+00,1.439461e+00,6.315017e-01,4.027293e-01,2.428076e+00,7.522194e-01,2.347238e+00,4.840380e-01,4.848866e-01,1.167503e+05,1.152136e+01,5.550367e+00,1.427102e+05,1.558229e+05,8.848481e+00,3.560263e+00,1.043695e+00,3.317342e+02,3.772171e-01,3.928351e-01,3.381996e-01,3.891512e-01
min,1.480000e+01,4.940000e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-01,1.000000e+00,2.012000e+03,1.000000e+00,0.000000e+00,6.491772e-04,-9.999500e-01,-1.539412e+03,9.676069e-03,4.587156e-02,1.078290e-02,2.185792e-03,0.000000e+00,2.312136e-02,0.000000e+00,0.000000e+00,1.307783e+05,-9.999520e-01,1.091529e-01,4.175000e+03,1.980733e+04,-9.998362e-01,2.514827e-01,2.834562e-02,-5.793991e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.087700e+05,1.305360e+05,6.907000e+01,7.869000e+01,2.000000e+00,6.000000e+00,1.800000e+00,2.900000e+01,2.015000e+03,4.000000e+00,1.000000e+00,7.042840e-01,-2.074267e-01,-4.198817e-01,4.900000e-01,4.615385e-01,1.818182e-01,2.000000e-01,1.000000e+00,1.333333e+00,0.000000e+00,0.000000e+00,1.602289e+05,-6.842105e-01,1.500000e+00,1.425000e+05,1.669275e+05,-4.877591e-01,9.322034e-01,1.603774e-01,-1.155705e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.803590e+05,2.067000e+05,1.063900e+02,1.165200e+02,4.000000e+00,1.500000e+01,3.000000e+00,5.600000e+01,2.018000e+03,6.000000e+00,2.000000e+00,9.252751e-01,8.947368e-03,-8.075965e-02,7.160494e-01,7.777778e-01,3.333333e-01,6.555556e-01,1.000000e+00,2.320830e+00,0.000000e+00,0.000000e+00,2.250787e+05,0.000000e+00,3.000000e+00,2.085075e+05,2.346268e+05,1.079172e-02,1.285714e+00,3.333333e-01,1.248181e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.030618e+05,3.297030e+05,1.696700e+02,1.802800e+02,1.200000e+01,3.900000e+01,5.000000e+00,1.000000e+02,2.021000e+03,9.000000e+00,2.000000e+00,1.097850e+00,2.884615e-01,8.912871e-02,1.000000e+00,1.072727e+00,5.555556e-01,2.000000e+00,2.000000e+00,3.785714e+00,1.000000e+00,1.000000e+00,3.140695e+05,2.500000e+00,6.235294e+00,3.082917e+05,3.356007e+05,9.829595e-01,2.166667e+00,6.666667e-01,1.718278e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.370925e+06,3.254070e+06,1.620572e+04,6.562700e+04,1.830000e+02,6.400000e+02,1.880000e+01,7.490000e+0

In [22]:
df_no_outliers.to_csv('/Users/jordonez/Desktop/Brainstation/CapstoneRepo/data/checkpoint5.csv')